* 1. Load the libries and Data For the project

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, LinearSVC,SVR
from sklearn.metrics import r2_score
import xgboost as xgb

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Modelling Helpers
from sklearn.preprocessing import Imputer , Normalizer , scale
from sklearn.cross_validation import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV


# Metrics


# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

store = pd.read_csv('./input/store.csv')
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

print(store.head())

/Users/roy/Documents/workplace/env/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


   Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1         c          a               1270.0                        9.0   
1      2         a          a                570.0                       11.0   
2      3         a          a              14130.0                       12.0   
3      4         c          c                620.0                        9.0   
4      5         a          a              29910.0                        4.0   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYear  \
0                    2008.0       0              NaN              NaN   
1                    2007.0       1             13.0           2010.0   
2                    2006.0       1             14.0           2011.0   
3                    2009.0       0              NaN              NaN   
4                    2015.0       0              NaN              NaN   

     PromoInterval  
0              NaN  
1  Jan,Apr,Jul,Oct  
2  Jan,Apr,

In [2]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()
    
def normalize(x):

    xmin = x.min()
    xmax = x.max()
    
    x = (x-xmin)/(xmax-xmin)

    return x

def rmspe_score(y_true,y_pred):
    return 'RMSPE',np.sqrt(np.mean((1-y_pred/y_true)**2))

def minmaxscaler(df):
    return 

def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )


In [6]:
#print train.isnull().sum()
#print store.isnull().sum()

print train.describe()
print '-----------------'
print store.describe()
#print store.columns
#index = store['CompetitionDistance'].index[store['CompetitionDistance'].apply(np.isnan)]
#print store.describe()
#store['CompetitionDistance'].fillna(0,inplace=True)
#store['PromoInterval'].fillna('no',inplace=True)


#print store[store['Promo2']==1].count()
#sns.countplot(data=store,x='StoreType',ax=ax[0,0])
#sns.countplot(data=store,x='Assortment',ax=ax[0,1])
#sns.countplot(data=store,x='PromoInterval',ax=ax[0,2])
#sns.countplot(data=store,x='')
#sns.distplot(store.Promo2)
#sns.distplot(store.CompetitionDistance,ax=ax[1,0])




              Store     DayOfWeek         Sales     Customers          Open  \
count  1.017209e+06  1.017209e+06  1.017209e+06  1.017209e+06  1.017209e+06   
mean   5.584297e+02  3.998341e+00  5.773819e+03  6.331459e+02  8.301067e-01   
std    3.219087e+02  1.997391e+00  3.849926e+03  4.644117e+02  3.755392e-01   
min    1.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    2.800000e+02  2.000000e+00  3.727000e+03  4.050000e+02  1.000000e+00   
50%    5.580000e+02  4.000000e+00  5.744000e+03  6.090000e+02  1.000000e+00   
75%    8.380000e+02  6.000000e+00  7.856000e+03  8.370000e+02  1.000000e+00   
max    1.115000e+03  7.000000e+00  4.155100e+04  7.388000e+03  1.000000e+00   

              Promo  SchoolHoliday  
count  1.017209e+06   1.017209e+06  
mean   3.815145e-01   1.786467e-01  
std    4.857586e-01   3.830564e-01  
min    0.000000e+00   0.000000e+00  
25%    0.000000e+00   0.000000e+00  
50%    0.000000e+00   0.000000e+00  
75%    1.000000e+00   0.00000

In [98]:
train_data = pd.merge(train,store,on='Store')

train_data.isnull().sum()



0          2015-07-31
1          2015-07-30
2          2015-07-29
3          2015-07-28
4          2015-07-27
5          2015-07-26
6          2015-07-25
7          2015-07-24
8          2015-07-23
9          2015-07-22
10         2015-07-21
11         2015-07-20
12         2015-07-19
13         2015-07-18
14         2015-07-17
15         2015-07-16
16         2015-07-15
17         2015-07-14
18         2015-07-13
19         2015-07-12
20         2015-07-11
21         2015-07-10
22         2015-07-09
23         2015-07-08
24         2015-07-07
25         2015-07-06
26         2015-07-05
27         2015-07-04
28         2015-07-03
29         2015-07-02
              ...    
1017179    2013-01-30
1017180    2013-01-29
1017181    2013-01-28
1017182    2013-01-27
1017183    2013-01-26
1017184    2013-01-25
1017185    2013-01-24
1017186    2013-01-23
1017187    2013-01-22
1017188    2013-01-21
1017189    2013-01-20
1017190    2013-01-19
1017191    2013-01-18
1017192    2013-01-17
1017193   

In [3]:
def build_features(features):
    features = pd.get_dummies(features,columns=['DayOfWeek','StoreType','Assortment','PromoInterval'])
    features['StateHoliday'].replace('a',1,inplace=True)
    features['StateHoliday'].replace('b',2,inplace=True)
    features['StateHoliday'].replace('c',3,inplace=True)
    features['StateHoliday'] =  pd.to_numeric(features['StateHoliday'], downcast='integer')
    features['StateHoliday'] = normalize(features["StateHoliday"])
    features['Date'] = pd.to_datetime(features['Date'])
    features['CompetitionOpenSinceMonth'] = pd.to_numeric(features['CompetitionOpenSinceMonth'], downcast='integer')
    features['CompetitionOpenSinceYear'] = pd.to_numeric(features['CompetitionOpenSinceYear'], downcast='integer')
    features['Month'] = features['Date'].map(lambda x: x.month)
    features['Year'] = features['Date'].map(lambda x: x.year)
    features['Week'] = features['Date'].map(lambda x: x.strftime("%V"))
    features['Week']=pd.to_numeric(features['Week'], downcast='integer')
    features["CompetitionMonthDuration"] = (12-features['CompetitionOpenSinceMonth']+features['Month'])+(features['Year']-features['CompetitionOpenSinceYear'])*12
    features["Promo2WeekDuration"] = (53-features['Promo2SinceWeek']+features['Week'])+(features['Year']-features['Promo2SinceYear'])*53
    features["CompetitionMonthDuration"] = normalize(features["CompetitionMonthDuration"])
    features["Promo2WeekDuration"] = normalize(features["Promo2WeekDuration"])
    features = features.drop(['CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2SinceWeek','Promo2SinceYear','Month','Year','Week','Date','Store'],axis=1)
    if 'Customers' in features.columns:
        features = features.drop(['Customers'],axis=1)
    features.fillna(0,inplace=True)
    
    return features

In [5]:
store["CompetitionDistance"] = normalize(store["CompetitionDistance"])

org_train_data = pd.merge(train,store,on='Store')
org_test_data = pd.merge(test,store,on='Store')

sales_train_raw = org_train_data[org_train_data['Sales']!=0]['Sales']
sales_train = np.log1p(sales_train_raw)

features_train_raw = org_train_data[org_train_data['Sales']!=0].drop('Sales',axis=1)

features_train = build_features(features_train_raw)

#print features.head()

# 将'features'和'val'数据切分成训练集和测试集
X_train, X_val, y_train, y_val = train_test_split(features_train, sales_train, test_size = 0.1,random_state = 42)

xgdmat=xgb.DMatrix(X_train,y_train)
params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.2,
          "max_depth": 12,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1
          }
num_trees = 50
#final_gb=xgb.train(params,xgdmat,num_trees,feval=rmspe_score)
#tesdmat=xgb.DMatrix(X_val)
#y_pred=final_gb.predict(tesdmat)

#print r2_score(y_val,y_pred)
#print rmspe_score(y_val,y_pred)

reg = LinearRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_val)

#print rmspe_score(y_val,y_pred)
print reg.score(X_val,y_val)

0.2411244309045144


In [ ]:
test = build_features(org_test_data)
test_train = test.drop(['Id'],axis=1)
test_result = final_gb.predict(xgb.DMatrix(test_train))
submission = pd.DataFrame({"Id": test['Id'], "Sales": np.expm1(test_result)})
submission.to_csv("my_submission.csv", index=False)

In [ ]:
# XGB feature importances
# Based on https://www.kaggle.com/mmueller/liberty-mutual-group-property-inspection-prediction/xgb-feature-importance-python/code
import operator

create_feature_map(features_train)
importance = final_gb.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))

df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()

featp = df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
fig_featp = featp.get_figure()
fig_featp.savefig('feature_importance_xgb.png', bbox_inches='tight', pad_inches=1)